In [5]:
import gymnasium as gym
from gymnasium.spaces import Discrete
from gymnasium.spaces import Box
import numpy as np

In [6]:
class TradingEnv(gym.Env):

    def __init__(self, data, normalized_data, budget=10000):
        self.budget = budget
        self.row_num = 0
        self.asset_allocation = 0.0
        self.data = data
        self.normalized_data = normalized_data
    
        self.action_space = Discrete(5)

        # observation space: Close, Volume, SMA Ratio, RSI, Bandwidth, Asset Allocation
        self.observation_space = Box(low=np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
            high=np.array([np.inf, np.inf, np.inf, 100.0, np.inf, 1.0]))


    def _get_obs(self):
        pass

    def _get_info(self):
        pass
        # return portfolio value at this point

    def reset(self, seed):
        super().reset(seed=seed)
        self.row_num = np.random.randint(0, int(0.8 * len(self.data)) - 2)
        # return obs, info

    def step(self, action):
        pass
        #return obs, rew, terminated, truncated, info

    def close(self):
        pass




IndentationError: expected an indented block (1076730244.py, line 16)